In [ ]:
import config
import json, csv, pprint, time, xlsxwriter
import numpy as np
import pandas as pd

from datetime import datetime
from binance.client import Client
from talib import RSI, PLUS_DI, DX, CCI, ADX, MINUS_DI,ATR ,SAR
from binance import BinanceSocketManager
from binance.enums import *
import bb
client = Client(config.API_KEY, config.API_SECRET)
import talib
import os
import math
import mplfinance
import matplotlib as mpf

In [ ]:
tradesm = bb.candlestick_backtest_maxprofit(SYMBOL=config.MATIC,START_DATE='1 week ago UTC',CANDLE_TIMEZONE='3m',RSI_TIMEZONE='15m')

In [ ]:
tradesn = bb.candlestick_backtest(SYMBOL=config.MATIC,START_DATE='1 week ago UTC',CANDLE_TIMEZONE='3m',RSI_TIMEZONE='15m')

In [ ]:
tradesm.head()
bb.set_opentime(tradesm)
bb.set_endtime(tradesm)

In [ ]:
dates = tradesm[['Open Time','End Time','TP','PNL','PNLQ']].copy()

In [ ]:

onem = client.get_historical_klines("XTZUSDT", Client.KLINE_INTERVAL_1MINUTE, "1 week ago UTC")
ohlc_onem = bb.get_ohlc_his(onem)

fivem = client.get_historical_klines("XTZUSDT", Client.KLINE_INTERVAL_5MINUTE, "1 week ago UTC")
ohlc_fivem = bb.get_ohlc_his(fivem)

threem = client.get_historical_klines("XTZUSDT", Client.KLINE_INTERVAL_3MINUTE, "1 week ago UTC")
ohlc_threem = bb.get_ohlc_his(threem)

fifm = client.get_historical_klines("XTZUSDT",Client.KLINE_INTERVAL_15MINUTE, "1 week ago UTC")
ohlc_fifm = bb.get_ohlc_his(fifm)




In [ ]:
d_p = dates.groupby('PNL')
d_p.get_group('Profit').to_csv('max_profit.csv')

In [ ]:
trades['PNLQ'].sum()
pnlm = trades.groupby('PNL')
len(trades)


In [ ]:
tradesn['PNLQ'].sum()
pnl=tradesn.groupby('PNL')
len(tradesn)

In [ ]:
open_onem = ohlc_onem[0]
high_onem = ohlc_onem[1]
low_onem = ohlc_onem[2]
close_onem = ohlc_onem[3]
#if open[i] == close[i] and close[i]> low[i]:
psar_onem = SAR(high_onem,low_onem)
open_threem = ohlc_threem[0]
high_threem = ohlc_threem[1]
low_threem = ohlc_threem[2]
close_threem = ohlc_threem[3]
#if open[i] == close[i] and close[i]> low[i]:
len(high_threem)




In [ ]:
def get_onem_buy_signal(high,close,psar):
    result = []
    for i in range(1,len(open)-1):
    
        if open[i] == high[i] and close[i]> low[i] and psar[i] > close[i]:
        
            result.append(i)
    return result
        
        

In [ ]:
df_fifm = pd.DataFrame(fifm, columns=config.FIELD_HISTORICAL)

In [ ]:
df_onem = pd.DataFrame(onem, columns=config.FIELD_HISTORICAL)

In [ ]:
df_threem = pd.DataFrame(threem, columns=config.FIELD_HISTORICAL)
df_fivem = pd.DataFrame(fivem, columns=config.FIELD_HISTORICAL)
bb.set_date(df_onem)
bb.set_date(df_fifm)
bb.set_date(df_threem)
bb.set_date(df_fivem)
'''df_onem.to_csv('oneminute.csv')
df_threem.to_csv('threeminute.csv')
df_fivem.to_csv('fiveminute.csv')
df_fifm.to_csv('fifminute.csv')'''

In [ ]:
df_onemf = df_onem.copy().iloc[:,:7]
df_onemf.head()

In [ ]:
len(fifm)

In [ ]:

df_threemf['Psar'] = psar_threem
df_threemf.dropna(inplace=True)
df_threemf.head(-5)

In [ ]:
df_fifmf = df_fifm.copy().iloc[:,:7]
rsi = RSI(df_fifmf['Close'])
df_fifmf['Rsi'] = rsi
df_fifmf.dropna(inplace=True)
df_fifmf.head(-5)

In [ ]:
df_fivemf = df_fivem.copy().iloc[:,:7]
rsif = RSI(df_fivem['Close'])
df_fivemf['Rsi'] = rsi
df_fivemf.dropna(inplace=True)

In [ ]:
permisson = False
res = []
trades = pd.DataFrame()
df_threem['Psar'] = df_threem['Psar'].astype(str)
for index, row in df_fifmf.iterrows():
    if not trades.empty:
        trades_status = trades.groupby(['Status'])
    #trades_status.get_group('Close')
    if row['Rsi'] > 50 : 
        permisson  = True
        time = row['Close Time']+1
        search_df = df_onemf.loc[(df_onemf['Open Time' ] >= time ) & (df_onemf['Open Time'] <= time+280000)]
        positions = search_df[(search_df['Open'] == search_df['High']) & (search_df['Close'] > search_df['Low'] ) & (search_df['Psar'] > search_df['Close'] )]
        positions.insert(8,'Status', 'Open')
        positions.insert(9,'Margin', 100)
        positions.insert(10,'End Time', '')
        positions['Close Price'] = np.nan
        positions['Ratio'] = np.nan
        positions.insert(13,'PNL', '')
        positions['PNLQ'] = np.nan
        positions
        trades = trades.append(positions)
        
        
        if not trades.empty:
            trades_status = trades.groupby(['Status'])
            try:
                if not trades_status.get_group('Open').empty:
                    for ind, ro in trades_status.get_group('Open').iterrows():
                        match_df = search_df.loc[(search_df['Open Time'] > ro['Open Time'])]
                        for i,r in match_df.iterrows():
                            if ro['Psar'] <= r['High']:
                                trades.at[ind,'End Time'] = r['Open Time']
                                trades.at[ind,'Close Price'] = ro['Psar']
                                trades.at[ind,'Ratio'] = float(ro['Psar'])/float(ro['Close'])
                                trades.at[ind,'PNL'] = 'Profit'
                                trades.at[ind,'PNLQ'] = ((float(ro['Psar'])/float(ro['Close']))*100) - 100
                                trades.at[ind,'Status'] = 'Close'
                                break
                        '''
                        ro['End Time'] = r['Open Time']
                        ro['Close Price'] = ro['Psar']
                        ro['Ratio'] = float(ro['Psar'])/float(ro['Close'])
                        ro['PNL'] = 'Profit'
                        ro['PNLQ'] = ((float(ro['Psar'])/float(ro['Close']))*100) - 100
                        ro['Status'] = 'Close'
                        '''
            except:
                print('All Position is closed')
                        
                       
    
    else:
        permission = False
        if not trades.empty:
            trades_status = trades.groupby(['Status'])
            try:
                if not trades_status.get_group('Open').empty:
                    for i, r in trades_status.get_group('Open').iterrows():

                        trades.at[i,'End Time'] = row['Close Time']
                        trades.at[i,'Close Price'] =  row['Close']
                        trades.at[i,'Ratio'] = float(row['Close'])/float(r['Close'])
                        trades.at[i,'PNL'] = 'Profit'
                        trades.at[i,'PNLQ'] = ((float(row['Close'])/float(r['Close']))*100) - 100
                        trades.at[i,'Status'] = 'Close'

                        '''
                        r['End Time'] = row['Close Time']
                        r['Ratio'] = float(row['Close'])/float(r['Close'])
                        r['Close Price'] = row['Close']
                        r['Status'] = 'Close'
                        r['PNLQ'] = ((float(row['Close'])/float(r['Close']))*100) - 100
                        '''
                        if row['Close'] > r['Close']:
                             trades.at[i,'PNL'] = 'Profit'
                        else:
                            trades.at[i,'PNL'] = 'Loss'
    
    
            except:
                print('All Position is Closed')
                        
                    
                    
        

In [ ]:
len(trades)

In [ ]:
d = 1634076000000 
search_df = df_onemf.loc[(df_onemf['Open Time' ] >= d ) & (df_onemf['Open Time'] <= d+840000)]

In [ ]:
'''signals_psar = []
for i in result:
    signals_psar.append(round(float(psar[i]),3))'''

In [ ]:
type(df_onemf['Psar'].iloc[0])

In [ ]:

df_onemf['Low'] = df_onemf['Low'].astype(float)

In [ ]:
'''z = 0
for index,psar in df_test.iloc[:].values:
    i = int(index) + 1
    
    for ind,high in td.iloc[i:,:].values:
        if float(high) >= float(psar):
            z = z + 1
            print(td['index'][index])
            print(ind)
            print("-----")
           
            
            
            break'''
    

In [ ]:
status = trades.groupby('Status')
closed_trades  = status.get_group('Close')


In [ ]:
pnl_grp = closed_trades.groupby('PNL')


In [ ]:
result = closed_trades['PNLQ'].sum()

In [ ]:
result

In [ ]:
len(closed_trades)

In [ ]:
pnl_grp.get_group('Loss').to_csv('Loss_Matic_1month.csv')

In [ ]:
pnl_grp.get_group('Profit').to_csv('Profit_Matic_1month.csv')

In [ ]:


tradesf = closed_trades.copy()

In [ ]:
date = tradesf['Open Time']
final_date = []
for time in date:
    time = time/1000
    readable = datetime.utcfromtimestamp(time)
    final_date.append(readable) 
tradesf['Open Time'] = final_date
    

In [ ]:
times = tradesf['End Time']
final_times = []
for d in times:
    d = d/1000
    readable = datetime.utcfromtimestamp(d)
    final_times.append(readable) 
tradesf['End Time'] = final_times

In [ ]:
tradesf[['Open Time','End Time','Close','Close Price','Ratio']].to_csv('trades_matic_1da.csv')

In [ ]:
trades[['Open Time','End Time','Close','Close Price','Ratio']].to_csv('trades_matic_.csv')

In [ ]:
tradesf

In [ ]:
df_oneplot = df_onemf.copy()
df_fiveplot = df_fivem.copy()

In [ ]:
df_oneplot['Volume'] = df_oneplot['Volume'].astype(float)
df_oneplot['High'] = df_oneplot['High'].astype(float)
df_oneplot['Low'] = df_oneplot['Low'].astype(float)
df_oneplot['Open'] = df_oneplot['Open'].astype(float)
df_oneplot['Close'] = df_oneplot['Close'].astype(float)
df_fiveplot['Volume'] = df_fiveplot['Volume'].astype(float)
df_fiveplot['High'] = df_fiveplot['High'].astype(float)
df_fiveplot['Low'] = df_fiveplot['Low'].astype(float)
df_fiveplot['Open'] = df_fiveplot['Open'].astype(float)
df_fiveplot['Close'] = df_fiveplot['Close'].astype(float)

In [ ]:
%matplotlib notebook


In [ ]:
df_fiveplot.to_csv('five.csv')

In [ ]:
mplfinance.plot(df_fiveplot,type='candle')

In [ ]:
%matplotlib inline

In [ ]:
mplfinance.plot(df_fiveplot,type='candle')

In [ ]:

matplotlib.pyplot.switch_backend()

In [ ]:
f = pd.read_csv('five.csv')

In [ ]:
fig = mpf.figure(figsize=(20,10)) 
ax1 = fig.add_subplot(2,1,1) 
ax2 = fig.add_subplot(2,1,2)



In [ ]:
permisson = False
res = []
trades = pd.DataFrame()
df_threemf['Psar'] = df_threemf['Psar'].astype(str)
for index, row in df_fifmf.iterrows():
    if not trades.empty:
        trades_status = trades.groupby(['Status'])
    #trades_status.get_group('Close')
    if row['Rsi'] > 50 : 
        permisson  = True
        time = row['Close Time']+1
        search_df = df_threemf.loc[(df_threemf['Open Time' ] >= time ) & (df_threemf['Open Time'] <= time+840000)]
        positions = search_df[(search_df['Open'] == search_df['High']) & (search_df['Close'] > search_df['Low'] ) & (search_df['Psar'] > search_df['Close'] )]
        positions.insert(8,'Status', 'Open')
        positions.insert(9,'Margin', 100)
        positions.insert(10,'End Time', '')
        positions['Close Price'] = np.nan
        positions['Ratio'] = np.nan
        positions.insert(13,'PNL', '')
        positions['PNLQ'] = np.nan
        positions
        trades = trades.append(positions)
        
        
        if not trades.empty:
            trades_status = trades.groupby(['Status'])
            try:
                if not trades_status.get_group('Open').empty:
                    for ind, ro in trades_status.get_group('Open').iterrows():
                        match_df = search_df.loc[(search_df['Open Time'] > ro['Open Time'])]
                        for i,r in match_df.iterrows():
                            if ro['Psar'] <= r['High']:
                                trades.at[ind,'End Time'] = r['Open Time']
                                trades.at[ind,'Close Price'] = ro['Psar']
                                trades.at[ind,'Ratio'] = float(ro['Psar'])/float(ro['Close'])
                                trades.at[ind,'PNL'] = 'Profit'
                                trades.at[ind,'PNLQ'] = ((float(ro['Psar'])/float(ro['Close']))*100) - 100
                                trades.at[ind,'Status'] = 'Close'
                                break
                        '''
                        ro['End Time'] = r['Open Time']
                        ro['Close Price'] = ro['Psar']
                        ro['Ratio'] = float(ro['Psar'])/float(ro['Close'])
                        ro['PNL'] = 'Profit'
                        ro['PNLQ'] = ((float(ro['Psar'])/float(ro['Close']))*100) - 100
                        ro['Status'] = 'Close'
                        '''
            except:
                print('All Position is closed')
                        
                       
    
    else:
        permission = False
        if not trades.empty:
            trades_status = trades.groupby(['Status'])
            try:
                if not trades_status.get_group('Open').empty:
                    for i, r in trades_status.get_group('Open').iterrows():

                        trades.at[i,'End Time'] = row['Close Time']
                        trades.at[i,'Close Price'] =  row['Close']
                        trades.at[i,'Ratio'] = float(row['Close'])/float(r['Close'])
                        trades.at[i,'PNL'] = 'Profit'
                        trades.at[i,'PNLQ'] = ((float(row['Close'])/float(r['Close']))*100) - 100
                        trades.at[i,'Status'] = 'Close'

                        '''
                        r['End Time'] = row['Close Time']
                        r['Ratio'] = float(row['Close'])/float(r['Close'])
                        r['Close Price'] = row['Close']
                        r['Status'] = 'Close'
                        r['PNLQ'] = ((float(row['Close'])/float(r['Close']))*100) - 100
                        '''
                        if row['Close'] > r['Close']:
                             trades.at[i,'PNL'] = 'Profit'
                        else:
                            trades.at[i,'PNL'] = 'Loss'
    
    
            except:
                print('All Position is Closed')
                        
                    
                    
                

In [ ]:
def candlestick_backtest(SYMBOL='',CANDLE_TIMEZONE='',RSI_TIMEZONE='',START_DATE='',END_DATE='',):
    interval = {
        '1m' : Client.KLINE_INTERVAL_1MINUTE ,
        '3m' : Client.KLINE_INTERVAL_3MINUTE ,
        '5m' : Client.KLINE_INTERVAL_5MINUTE ,
        '15m': Client.KLINE_INTERVAL_15MINUTE ,
        '30m': Client.KLINE_INTERVAL_30MINUTE,
        '1h': Client.KLINE_INTERVAL_1HOUR,
        '4h': Client.KLINE_INTERVAL_4HOUR,
        '1d': Client.KLINE_INTERVAL_1DAY,
        '1w': Client.KLINE_INTERVAL_1WEEK,
    }
    SEARCH_RATE=840000
    if RSI_TIMEZONE == '15m' and CANDLE_TIMEZONE =='1m':
        SEARCH_RATE = 840000
    if RSI_TIMEZONE == '15m' and CANDLE_TIMEZONE =='3m':
        SEARCH_RATE = 840000
    if RSI_TIMEZONE == '5m' and CANDLE_TIMEZONE =='1m':
        SEARCH_RATE = 240000
    if RSI_TIMEZONE == '5m' and CANDLE_TIMEZONE =='3m':
        SEARCH_RATE = 240000
    if END_DATE == '':
        candle_data = client.get_historical_klines(SYMBOL, interval[CANDLE_TIMEZONE],START_DATE)
        rsi_data = client.get_historical_klines(SYMBOL, interval[RSI_TIMEZONE],START_DATE)
    else:
        candle_data = client.get_historical_klines(SYMBOL, interval[CANDLE_TIMEZONE],START_DATE,END_DATE)
        rsi_data = client.get_historical_klines(SYMBOL, interval[RSI_TIMEZONE],START_DATE,END_DATE)
    
    
        
    
    df_candle = pd.DataFrame(candle_data, columns=config.FIELD_HISTORICAL)
    df_rsi = pd.DataFrame(rsi_data, columns=config.FIELD_HISTORICAL)
    bb.set_date(df_candle)
    bb.set_date(df_rsi)
    df_candle = df_candle.copy().iloc[:,:7]
    df_rsi = df_rsi.copy().iloc[:,:7]
    
    psar_candle = SAR(df_candle['High'],df_candle['Low'])
    rsi = RSI(df_rsi['Close'])
    
    
    df_candle['PSAR'] = psar_candle
    df_candle.dropna(inplace=True)
    
    df_rsi['RSI'] = rsi
    df_rsi.dropna(inplace=True)
    
    permisson = False
    res = []
    trades = pd.DataFrame()
    df_candle['PSAR'] = df_candle['PSAR'].astype(str)
    for index, row in df_rsi.iterrows():
        
        if not trades.empty:
            trades_status = trades.groupby(['Status'])
        #trades_status.get_group('Close')
        if row['RSI'] > 50 : 
            permisson  = True
            time = row['Close Time']+1
            search_df = df_candle.loc[(df_candle['Open Time' ] >= time ) & (df_candle['Open Time'] <= time+SEARCH_RATE)]
            positions = search_df[(search_df['Open'] == search_df['High']) & (search_df['Close'] > search_df['Low'] ) & (search_df['PSAR'] > search_df['Close'] )]
            positions.insert(8,'Status', 'Open')
            positions.insert(9,'Margin', 100)
            positions.insert(10,'End Time', '')
            positions['Close Price'] = np.nan
            positions['Ratio'] = np.nan
            positions.insert(13,'PNL', '')
            positions['PNLQ'] = np.nan
            positions
            trades = trades.append(positions)


            if not trades.empty:
                trades_status = trades.groupby(['Status'])
                try:
                    if not trades_status.get_group('Open').empty:
                        for ind, ro in trades_status.get_group('Open').iterrows():
                            match_df = search_df.loc[(search_df['Open Time'] > ro['Open Time'])]
                            for i,r in match_df.iterrows():
                                if ro['PSAR'] <= r['High']:
                                    trades.at[ind,'End Time'] = r['Open Time']
                                    trades.at[ind,'Close Price'] = ro['PSAR']
                                    trades.at[ind,'Ratio'] = float(ro['PSAR'])/float(ro['Close'])
                                    trades.at[ind,'PNL'] = 'Profit'
                                    trades.at[ind,'PNLQ'] = ((float(ro['PSAR'])/float(ro['Close']))*100) - 100
                                    trades.at[ind,'Status'] = 'Close'
                                    break
                            '''
                            ro['End Time'] = r['Open Time']
                            ro['Close Price'] = ro['Psar']
                            ro['Ratio'] = float(ro['Psar'])/float(ro['Close'])
                            ro['PNL'] = 'Profit'
                            ro['PNLQ'] = ((float(ro['Psar'])/float(ro['Close']))*100) - 100
                            ro['Status'] = 'Close'
                            '''
                except:
                    print('All Position is closed')



        else:
            permission = False
            if not trades.empty:
                trades_status = trades.groupby(['Status'])
                try:
                    if not trades_status.get_group('Open').empty:
                        for i, r in trades_status.get_group('Open').iterrows():

                            trades.at[i,'End Time'] = row['Close Time']
                            trades.at[i,'Close Price'] =  row['Close']
                            trades.at[i,'Ratio'] = float(row['Close'])/float(r['Close'])
                            trades.at[i,'PNL'] = 'Profit'
                            trades.at[i,'PNLQ'] = ((float(row['Close'])/float(r['Close']))*100) - 100
                            trades.at[i,'Status'] = 'Close'

                            '''
                            r['End Time'] = row['Close Time']
                            r['Ratio'] = float(row['Close'])/float(r['Close'])
                            r['Close Price'] = row['Close']
                            r['Status'] = 'Close'
                            r['PNLQ'] = ((float(row['Close'])/float(r['Close']))*100) - 100
                            '''
                            if row['Close'] > r['Close']:
                                 trades.at[i,'PNL'] = 'Profit'
                            else:
                                trades.at[i,'PNL'] = 'Loss'


                except:
                    print('All Position is Closed')

                    
    
    
    df = trades.groupby('Status')
    
    return df.get_group('Close')

    
    

In [ ]:
len(config.ARRAY)

In [ ]:
sum = 0 ;
for coin in config.ARRAY : 
    trades = bb.candlestick_backtest_maxprofit(SYMBOL=coin,START_DATE='1 week ago UTC',CANDLE_TIMEZONE='3m',RSI_TIMEZONE='15m')
    sum = sum + float(trades['PNLQ'].sum())
    
    

In [ ]:
sum

In [ ]:

trades = bb.candlestick_backtest(SYMBOL=config.MATIC,START_DATE='1 week ago UTC',CANDLE_TIMEZONE='3m',RSI_TIMEZONE='15m')

In [ ]:
trades.head()

In [ ]:
trades['PNL']

In [ ]:
trades.head()